# Document Templates Setup
## Purpose: Create sample document templates for testing

This notebook creates sample document templates to enable proper testing of the Document Templates API.
The main template testing failed because no templates were configured in the vault.

**Prerequisites:**
- Valid VeevaVault authentication
- Document types configured in vault
- File staging capabilities
- Template creation permissions

In [2]:
# Import required libraries
import sys
import os
import tempfile
from pathlib import Path

# Add the parent directory to sys.path to import our modules
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

from veevavault.client.vault_client import VaultClient
from test_credentials import TEST_VAULTS, DEFAULT_VAULT

# Get vault configuration
vault_config = TEST_VAULTS[DEFAULT_VAULT]
VAULT_URL = vault_config["URL"]
VAULT_USERNAME = vault_config["username"]
VAULT_PASSWORD = vault_config["password"]

print("📦 Libraries imported successfully")
print(f"🔗 Target Vault: {VAULT_URL}")
print(f"👤 Username: {VAULT_USERNAME}")

📦 Libraries imported successfully
🔗 Target Vault: https://vv-consulting-michael-mastermind.veevavault.com/
👤 Username: veevatools@vv-consulting.com


In [6]:
# Initialize client and authenticate
print("🔐 Initializing VeevaVault client...")
client = VaultClient()

# Set vault URL in client before authentication
client.vaultURL = VAULT_URL.rstrip('/')  # Remove trailing slash if present

print("🔑 Authenticating...")
# Use AuthenticationService directly for proper authentication
from veevavault.services.authentication import AuthenticationService
auth_service = AuthenticationService(client)

# Extract DNS from URL 
VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")

auth_response = auth_service.authenticate_with_username_password(
    username=VAULT_USERNAME,
    password=VAULT_PASSWORD,
    vaultDNS=VAULT_DNS
)

if auth_response.get('responseStatus') == 'SUCCESS':
    session_id = auth_response['sessionId']
    vault_id = auth_response.get('vaultId', 'Unknown')
    print(f"✅ Authentication successful!")
    print(f"🆔 Session ID: {session_id[:20]}...")
    print(f"🏢 Vault ID: {vault_id}")
    auth_success = True
else:
    print(f"❌ Authentication failed: {auth_response}")
    auth_success = False

🔐 Initializing VeevaVault client...
🔑 Authenticating...
✅ Authentication successful!
🆔 Session ID: 5DF6BFDAF1956FC4E5D8...
🏢 Vault ID: 92425


In [9]:
# Test: Check available document types for templates
if auth_success:
    print("📋 Testing document types discovery...")
    
    # Get document types
    types_response = client.api_call("api/v25.2/metadata/objects/documents/types")
    
    if types_response.get('responseStatus') == 'SUCCESS':
        types_data = types_response.get('types', [])
        print(f"✅ Found {len(types_data)} document types")
        
        # Display first few types for reference
        print("\\n📝 Available Document Types:")
        for i, doc_type in enumerate(types_data[:5]):  # Show first 5
            type_label = doc_type.get('label', 'No label')
            # Extract type name from URL
            type_url = doc_type.get('value', '')
            type_name = type_url.split('/')[-1] if type_url else 'Unknown'
            print(f"  {i+1}. {type_label} ({type_name})")
        
        if len(types_data) > 5:
            print(f"  ... and {len(types_data) - 5} more")
            
        # Save first available type for template creation
        if types_data:
            first_type_url = types_data[0].get('value', '')
            first_type = first_type_url.split('/')[-1] if first_type_url else None
            print(f"\\n🎯 Selected type for template: {first_type}")
        
    else:
        print(f"❌ Failed to get document types: {types_response}")
        types_data = []

📋 Testing document types discovery...
✅ Found 6 document types
\n📝 Available Document Types:
  1. Staged (staged__v)
  2. Mythic Reliquary (mythic_reliquary__c)
  3. Secret Scrolls (secret_scrolls__c)
  4. 22R1 Release Doc Type (22r1_release_doc_type__c)
  5. Base Document (base_document__v)
  ... and 1 more
\n🎯 Selected type for template: staged__v


In [10]:
# Test: Create a simple template file for upload
if auth_success and types_data:
    print("📄 Creating sample template file...")
    
    # Create a simple template file content
    template_content = """
Sample Document Template
========================

This is a sample document template created for testing purposes.

Document Information:
- Title: [DOCUMENT_TITLE]
- Date: [DOCUMENT_DATE]
- Author: [DOCUMENT_AUTHOR]
- Version: [DOCUMENT_VERSION]

Content:
[DOCUMENT_CONTENT]

Footer:
This document was created from a VeevaVault template.
"""
    
    # Create temporary file
    with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False) as temp_file:
        temp_file.write(template_content)
        temp_file_path = temp_file.name
    
    print(f"✅ Created template file: {temp_file_path}")
    print(f"📊 File size: {len(template_content)} characters")

📄 Creating sample template file...
✅ Created template file: /var/folders/rq/8j277nys17b3_b201kqg50680000gn/T/tmpkq4l83gc.txt
📊 File size: 340 characters


In [11]:
# Test: Check current templates (should be empty)
if auth_success:
    print("🔍 Checking existing templates...")
    
    current_templates = client.api_call("api/v25.2/objects/documents/templates")
    
    if current_templates.get('responseStatus') == 'SUCCESS':
        template_count = len(current_templates.get('data', []))
        print(f"📊 Current template count: {template_count}")
        
        if template_count == 0:
            print("✅ Confirmed: No existing templates (as expected)")
        else:
            print("⚠️ Templates already exist:")
            for template in current_templates.get('data', [])[:3]:
                print(f"  - {template.get('name__v', 'Unknown')}: {template.get('label__v', 'No label')}")
    else:
        print(f"❌ Could not check templates: {current_templates}")

🔍 Checking existing templates...
📊 Current template count: 1
⚠️ Templates already exist:
  - basic_template__c: Basic Template


In [12]:
# Test: Create template using API
if auth_success and types_data and 'temp_file_path' in locals():
    print("🛠️ Attempting to create document template...")
    
    try:
        # Prepare template data
        template_data = {
            'label__v': 'VeevaTools Test Template',
            'type__v': first_type,
            'active__v': 'true'
        }
        
        # Try to upload template
        with open(temp_file_path, 'rb') as template_file:
            files = {'file': ('test_template.txt', template_file, 'text/plain')}
            
            response = client.api_call(
                "api/v25.2/objects/documents/templates",
                method="POST",
                data=template_data,
                files=files
            )
        
        print(f"📤 Template creation response: {response}")
        
        if response.get('responseStatus') == 'SUCCESS':
            print("✅ Template created successfully!")
            template_name = response.get('name', 'Unknown')
            print(f"🏷️ Template name: {template_name}")
            
            # Verify template was created
            verify_response = client.api_call("api/v25.2/objects/documents/templates")
            if verify_response.get('responseStatus') == 'SUCCESS':
                new_count = len(verify_response.get('data', []))
                print(f"📊 Template count after creation: {new_count}")
                
        else:
            print(f"❌ Template creation failed: {response}")
            error_type = response.get('errors', [{}])[0].get('type', 'Unknown')
            error_message = response.get('errors', [{}])[0].get('message', 'No message')
            print(f"🔍 Error type: {error_type}")
            print(f"💬 Error message: {error_message}")
            
    except Exception as e:
        print(f"💥 Exception during template creation: {str(e)}")
    
    finally:
        # Clean up temporary file
        if os.path.exists(temp_file_path):
            os.unlink(temp_file_path)
            print(f"🧹 Cleaned up temporary file")

🛠️ Attempting to create document template...
📤 Template creation response: {'responseStatus': 'SUCCESS', 'name__v': 'veevatools_test_template__c'}
✅ Template created successfully!
🏷️ Template name: Unknown
📊 Template count after creation: 2
🧹 Cleaned up temporary file


In [13]:
# Test: Alternative approach - Check for file staging
if auth_success:
    print("📁 Checking file staging capabilities...")
    
    # Check file staging endpoint
    staging_response = client.api_call("api/v25.2/services/file_staging")
    
    print(f"📋 File staging response: {staging_response}")
    
    if staging_response.get('responseStatus') == 'SUCCESS':
        print("✅ File staging available")
    else:
        print("⚠️ File staging may not be available or configured")

📁 Checking file staging capabilities...


Exception: HTTP error occurred: 404 Client Error: Not Found for url: https://vv-consulting-michael-mastermind.veevavault.com/api/v25.2/services/file_staging

In [14]:
# Test: Check template metadata requirements
if auth_success:
    print("🔍 Checking template metadata requirements...")
    
    metadata_response = client.api_call("api/v25.2/metadata/objects/documents/templates")
    
    if metadata_response.get('responseStatus') == 'SUCCESS':
        print("✅ Template metadata retrieved")
        
        # Check required fields
        fields = metadata_response.get('fields', [])
        required_fields = [field for field in fields if field.get('required', False)]
        
        print(f"📊 Total fields: {len(fields)}")
        print(f"🔒 Required fields: {len(required_fields)}")
        
        if required_fields:
            print("\n📋 Required fields for template creation:")
            for field in required_fields[:5]:  # Show first 5
                field_name = field.get('name', 'Unknown')
                field_type = field.get('type', 'Unknown')
                print(f"  - {field_name} ({field_type})")
    else:
        print(f"❌ Could not get template metadata: {metadata_response}")

🔍 Checking template metadata requirements...
✅ Template metadata retrieved
📊 Total fields: 0
🔒 Required fields: 0


## 🎯 Setup Analysis Summary

This notebook tested various approaches to create document templates:

1. **Document Types Discovery** - Check available types for template association
2. **Template File Creation** - Create sample template content
3. **API Template Creation** - Attempt to create template via API
4. **File Staging Check** - Verify file upload capabilities
5. **Metadata Analysis** - Understand template requirements

**Next Steps:**
- If template creation succeeded, re-run the main templates testing notebook
- If failed, check vault permissions and configuration requirements
- Consider alternative approaches like using existing documents as template sources